# Connery Action Tool

Using this tool, you can integrate individual Connery Action into your LangChain agents and chains.

If you want to use several Connery Actions in your agent,
check out the [Connery Toolkit](/docs/integrations/toolkits/connery) documentation.

## What is Connery?

Connery is an open-source plugin infrastructure for AI.

With Connery, you can easily create a custom plugin with a set of actions and seamlessly integrate them into your LangChain agents and chains.
Connery will handle the rest: runtime, authorization, secret management, access management, audit logs, and other vital features.
Additionally, Connery and our community offer a wide range of ready-to-use open-source plugins for your convenience.

Learn more about Connery:

- GitHub: https://github.com/connery-io/connery-platform
- Documentation: https://docs.connery.io

## Prerequisites

To use Connery Actions in your LangChain agents, you need to do some preparation:

1. Set up the Connery runner using the [Quickstart](https://docs.connery.io/docs/platform/quick-start/) guide.
2. Install all the plugins with the actions you want to use in your agent.
3. Set environment variables `CONNERY_RUNNER_URL` and `CONNERY_RUNNER_API_KEY` so the toolkit can communicate with the Connery Runner.

## Example

In the example below, we fetch action by its ID from the Connery Runner and then call it with the specified parameters.
We use the ID of the **Send email** action from the [Gmail](https://github.com/connery-io/gmail) plugin.
You can find action IDs in the Connery Runner.

NOTE: Connery Action is a structured tool, so you can only use it in the agents supporting structured tools.

In [14]:
import os
from langchain_community.tools.connery import ConneryService
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent

# Specify your Connery Runner credentials.
os.environ["CONNERY_RUNNER_URL"] = ""
os.environ["CONNERY_RUNNER_API_KEY"] = ""

# Specify OpenAI API key.
os.environ["OPENAI_API_KEY"] = ""

# Specify your email address to receive the emails from examples below.
recepient_email = "test@example.com"

connery_service = ConneryService()
send_email_action = connery_service.get_action("CABC80BB79C15067CA983495324AE709")

ValidationError: 1 validation error for ConneryService
__root__
  Did not find runner_url, please add an environment variable `CONNERY_RUNNER_URL` which contains it, or pass  `runner_url` as a named parameter. (type=value_error)

In [5]:
# Check the action's input schema.
print(send_email_action.get_schema_json())

{
  "title": "InputSchema",
  "type": "object",
  "properties": {
    "recipient": {
      "title": "Email Recipient",
      "description": "Email Recipient: Email address of the email recipient.",
      "type": "string"
    },
    "subject": {
      "title": "Email Subject",
      "description": "Email Subject: Subject of the email.",
      "type": "string"
    },
    "body": {
      "title": "Email Body",
      "description": "Email Body: Body of the email.",
      "type": "string"
    }
  },
  "required": [
    "recipient",
    "subject",
    "body"
  ]
}


Run the action manually:

In [ ]:
send_email_action.run({
    "recipient": recepient_email, 
    "subject": "Test email",
    "body": "This is a test email sent from Connery.",
})

Run the action using the OpenAI Functions agent:

In [8]:
llm = ChatOpenAI(temperature=0)
agent = initialize_agent(
    [send_email_action], 
    llm, 
    AgentType.OPENAI_FUNCTIONS, 
    verbose=True
)

agent.run(f"Send an email to the {recepient_email} and say that I will be late for the meeting.")



> Entering new AgentExecutor chain...

Invoking: `CABC80BB79C15067CA983495324AE709` with `{'recipient': 'test@example.com', 'subject': 'Late for Meeting', 'body': 'Dear Team,\n\nI wanted to inform you that I will be late for the meeting today. I apologize for any inconvenience caused. Please proceed with the meeting without me and I will join as soon as I can.\n\nBest regards,\n[Your Name]'}`


{'messageId': '<74441e82-43a3-322b-d41a-0f96357201e9@gmail.com>'}I have sent an email to test@example.com with the subject "Late for Meeting" and the following message:

Dear Team,

I wanted to inform you that I will be late for the meeting today. I apologize for any inconvenience caused. Please proceed with the meeting without me and I will join as soon as I can.

Best regards,
[Your Name]

> Finished chain.


'I have sent an email to test@example.com with the subject "Late for Meeting" and the following message:\n\nDear Team,\n\nI wanted to inform you that I will be late for the meeting today. I apologize for any inconvenience caused. Please proceed with the meeting without me and I will join as soon as I can.\n\nBest regards,\n[Your Name]'